# Run the prompts and report results for zero shot setting

## cause and effect sentences

In [1]:
import os
import openai
import pandas as pd
from tqdm import tqdm

# Load your API key from an environment variable or secret management service
openai.api_key = os.getenv("OPENAI_KEY")

In [2]:
gpt3_model = "text-davinci-001"
#gpt3_model = "text-curie-001"
#gpt3_model = "text-babbage-001"
#gpt3_model = "text-ada-001"

In [34]:
test_prompt = "My car got dirty. I washed the car. Question: Which sentence is the cause of the other? Answer: "
print(test_prompt)

My car got dirty. I washed the car. Question: Which sentence is the cause of the other? Answer: 


In [35]:
# test run
response = openai.Completion.create(engine=gpt3_model, prompt=test_prompt, max_tokens=20, temperature=0)
print(response["choices"][0]["text"])



The sentence "I washed the car" is the cause of the other sentence "my car


In [5]:
# load dataframe

df_ce_two_sentences = pd.read_csv("data/bigbench_csvs/ce_two_sentences.csv")
df_ce_two_sentences.head()

,prompts,correct_answers
0,The child hurt their knee. The child started c...,The child hurt their knee.
1,The child started crying. The child hurt their...,The child hurt their knee.
2,My car got dirty. I washed the car.,My car got dirty.
3,I washed the car. My car got dirty.,My car got dirty.
4,Someone fainted. Someone called 911.,Someone fainted.


In [6]:
cause_question = "\nQuestion: Which sentence is the cause of the other?\nAnswer: "
effect_question = "\nQuestion: Which sentence is the effect of the other?\nAnswer: "

In [7]:
# run and populate the entire dataframe
cause_answers = []
effect_answers = []

for index, row in tqdm(df_ce_two_sentences.iterrows()):
    prompt_ = row["prompts"]
    # cause
    prompt_cause = prompt_ + cause_question 
    response_cause = openai.Completion.create(engine=gpt3_model, prompt=prompt_cause, max_tokens=20, temperature=0)
    rc_ = response_cause["choices"][0]["text"]
    cause_answers.append(rc_)
    # effect
    prompt_effect = prompt_ + effect_question 
    response_effect = openai.Completion.create(engine=gpt3_model, prompt=prompt_effect, max_tokens=20, temperature=0)
    re_ = response_effect["choices"][0]["text"]
    effect_answers.append(re_)

102it [02:39,  1.56s/it]


In [8]:
# add results to the dataframe and save

df_ce_two_sentences["cause_answers"] = cause_answers
df_ce_two_sentences["effect_answers"] = effect_answers
df_ce_two_sentences.head()

,prompts,correct_answers,cause_answers,effect_answers
0,The child hurt their knee. The child started c...,The child hurt their knee.,\n\nThe child's crying caused them to hurt the...,\n\nThe child started crying is the effect of ...
1,The child started crying. The child hurt their...,The child hurt their knee.,\n\nThe child started crying because they hurt...,\n\nThe child started crying because they hurt...
2,My car got dirty. I washed the car.,My car got dirty.,"\n\nThe sentence ""I washed the car"" is the cau...","\n\nThe sentence ""I washed the car"" is the eff..."
3,I washed the car. My car got dirty.,My car got dirty.,"\n\nThe sentence ""I washed the car"" is the cau...","\n\nThe sentence ""I washed the car"" is the eff..."
4,Someone fainted. Someone called 911.,Someone fainted.,"\n\nThe sentence ""Someone fainted"" is the caus...","\n\nThe sentence ""Someone fainted"" is the effe..."


In [9]:
path = "data/bigbench_results/ce_two_sentences_results_{}.csv".format(gpt3_model)
df_ce_two_sentences.to_csv(path, index=False)

## which sentence is more plausible?

In [10]:
# load dataframe

df_ce_one_sentence = pd.read_csv("data/bigbench_csvs/ce_one_sentence.csv")
df_ce_one_sentence.head()

,prompts,correct_answers
0,The child started crying because the child hur...,The child started crying because the child hur...
1,The child hurt their knee because the child st...,The child started crying because the child hur...
2,I washed the car because my car got dirty. My ...,I washed the car because my car got dirty.
3,My car got dirty because i washed the car. I w...,I washed the car because my car got dirty.
4,Someone called 911 because someone fainted. So...,Someone called 911 because someone fainted.


In [31]:
test_prompt2 = "Question: Which sentence gets cause and effect right?\nI washed the car because my car got dirty. My car got dirty because I washed the car. Answer: "
print(test_prompt2)

Question: Which sentence gets cause and effect right?
I washed the car because my car got dirty. My car got dirty because I washed the car. Answer: 


In [12]:
# test run
response2 = openai.Completion.create(engine=gpt3_model, prompt=test_prompt2, max_tokens=25, temperature=0)
print(response2["choices"][0]["text"])



The sentence that gets cause and effect right is "My car got dirty because I washed the car."


In [13]:
#question = "\nWhich sentence is more plausible?"
#question = "\nWhich sentence is more logical?"
question = "\nQuestion: Which sentence gets cause and effect right?\nAnswer: "

In [14]:
# run and populate the entire dataframe
answers2 = []

for index, row in tqdm(df_ce_one_sentence.iterrows()):
    prompt_ = row["prompts"]
    # cause
    prompt = prompt_ + question
    #print(prompt)
    response = openai.Completion.create(engine=gpt3_model, prompt=prompt, max_tokens=25, temperature=0)
    response_text = response["choices"][0]["text"]
    #print(response_text)
    answers2.append(response_text)

102it [01:20,  1.27it/s]


In [15]:
# add results to the dataframe and save

df_ce_one_sentence["answers"] = answers2
df_ce_one_sentence.head()

,prompts,correct_answers,answers
0,The child started crying because the child hur...,The child started crying because the child hur...,\n\nThe child started crying because the child...
1,The child hurt their knee because the child st...,The child started crying because the child hur...,\n\nThe child started crying because the child...
2,I washed the car because my car got dirty. My ...,I washed the car because my car got dirty.,\n\nThe sentence that gets cause and effect ri...
3,My car got dirty because i washed the car. I w...,I washed the car because my car got dirty.,\n\nThe sentence that gets cause and effect ri...
4,Someone called 911 because someone fainted. So...,Someone called 911 because someone fainted.,\n\nSomeone called 911 because someone fainted.


In [16]:
path = "data/bigbench_results/ce_one_sentence_results_{}.csv".format(gpt3_model)
df_ce_one_sentence.to_csv(path, index=False)

## Toy sentences 3 colors

In [17]:
df_toy_problem_3c = pd.read_csv("data/toy_problem_3/toy_problem_3c.csv")
df_toy_problem_3c.head()

,sequence,switched,first_color,second_color,final_color
0,The blue ball hit the red ball. The red ball h...,False,blue,red,green
1,The red ball hit the green ball. The blue ball...,True,blue,red,green
2,The blue ball hit the red ball. The red ball h...,False,blue,red,brown
3,The red ball hit the brown ball. The blue ball...,True,blue,red,brown
4,The blue ball hit the red ball. The red ball h...,False,blue,red,purple


In [32]:
test_prompt_3c = "The blue ball hit the red ball. The red ball hit the green ball. The green ball fell in the hole. Question: Which ball started the chain? Answer: "
test_prompt_3c_switched = "The red ball hit the green ball. The blue ball hit the red ball. The green ball fell in the hole. Question: Which ball started the chain? Answer: "
print(test_prompt_3c)
print("\n")
print(test_prompt_3c_switched)

The blue ball hit the red ball. The red ball hit the green ball. The green ball fell in the hole. Question: Which ball started the chain? Answer: 


The red ball hit the green ball. The blue ball hit the red ball. The green ball fell in the hole. Question: Which ball started the chain? Answer: 


In [19]:
# test run
response_3c = openai.Completion.create(engine=gpt3_model, prompt=test_prompt_3c, max_tokens=15, temperature=0)
print(response_3c["choices"][0]["text"])
# test run
response_3c_switched = openai.Completion.create(engine=gpt3_model, prompt=test_prompt_3c_switched, max_tokens=15, temperature=0)
print(response_3c_switched["choices"][0]["text"])



The blue ball started the chain.


The blue ball started the chain.


In [20]:
question1 = "\nQuestion: Which ball started the chain?\nAnswer: "
question2 = "\nQuestion: Which ball was second in the chain?\nAnswer: "
question3 = "\nQuestion: Which ball fell into the hole?\nAnswer: "

In [21]:
# run and populate the entire dataframe
answers_c3_q1 = []
answers_c3_q2 = []
answers_c3_q3 = []

for index, row in tqdm(df_toy_problem_3c.iterrows()):
    prompt_q1 = row["sequence"] +  question1
    prompt_q2 = row["sequence"] +  question2
    prompt_q3 = row["sequence"] +  question3
    response_q1 = openai.Completion.create(engine=gpt3_model, prompt=prompt_q1, max_tokens=15, temperature=0)
    response_text_q1 = response_q1["choices"][0]["text"]
    response_q2 = openai.Completion.create(engine=gpt3_model, prompt=prompt_q2, max_tokens=15, temperature=0)
    response_text_q2 = response_q2["choices"][0]["text"]
    response_q3 = openai.Completion.create(engine=gpt3_model, prompt=prompt_q3, max_tokens=15, temperature=0)
    response_text_q3 = response_q3["choices"][0]["text"]
#    print(response_text_q1)
#    print(response_text_q2)
#    print(response_text_q3)
    answers_c3_q1.append(response_text_q1)
    answers_c3_q2.append(response_text_q2)
    answers_c3_q3.append(response_text_q3)

70it [02:11,  1.88s/it]


In [22]:
df_toy_problem_3c["answers_start_chain"] = answers_c3_q1
df_toy_problem_3c["answers_second_chain"] = answers_c3_q2
df_toy_problem_3c["answers_into_hole"] = answers_c3_q3

path = "data/toy_problem_3_results/toy_problem_3c_results_{}.csv".format(gpt3_model)
df_toy_problem_3c.to_csv(path, index=False)
df_toy_problem_3c.head()

,sequence,switched,first_color,second_color,final_color,answers_start_chain,answers_second_chain,answers_into_hole
0,The blue ball hit the red ball. The red ball h...,False,blue,red,green,\n\nThe blue ball started the chain.,\n\nThe blue ball was second in the chain.,\n\nThe green ball fell into the hole.
1,The red ball hit the green ball. The blue ball...,True,blue,red,green,\n\nThe blue ball started the chain.,\n\nThe blue ball was second in the chain.,\n\nThe green ball fell into the hole.
2,The blue ball hit the red ball. The red ball h...,False,blue,red,brown,\n\nThe blue ball started the chain.,\n\nThe blue ball was second in the chain.,\n\nThe brown ball fell into the hole.
3,The red ball hit the brown ball. The blue ball...,True,blue,red,brown,\n\nThe blue ball started the chain.,\n\nThe blue ball was second in the chain.,\n\nThe brown ball fell into the hole.
4,The blue ball hit the red ball. The red ball h...,False,blue,red,purple,\n\nThe blue ball started the chain.,\n\nThe blue ball was second in the chain.,\n\nThe purple ball fell into the hole.


## Nonsense words

In [33]:
test_prompt_3nonsense = "The schleep hit the blubb. The blubb hit the baz. The baz fell in the hole. Question: What started the chain? Answer: "
test_prompt_3nonsense_switched = "The blubb hit the baz. The schleep hit the blubb. The baz fell in the hole. Question: What started the chain? Answer: "
print(test_prompt_3nonsense)
print("\n")
print(test_prompt_3nonsense_switched)

The schleep hit the blubb. The blubb hit the baz. The baz fell in the hole. Question: What started the chain? Answer: 


The blubb hit the baz. The schleep hit the blubb. The baz fell in the hole. Question: What started the chain? Answer: 


In [24]:
response_3nonsense = openai.Completion.create(engine=gpt3_model, prompt=test_prompt_3nonsense, max_tokens=15, temperature=0)
print(response_3c["choices"][0]["text"])
response_3nonsense_switched = openai.Completion.create(engine=gpt3_model, prompt=test_prompt_3nonsense_switched, max_tokens=15, temperature=0)
print(response_3nonsense_switched["choices"][0]["text"])



The blue ball started the chain.


The blubb hit the baz.


In [25]:
df_toy_problem_3nonsense = pd.read_csv("data/toy_problem_3/toy_problem_3nonsense.csv")
df_toy_problem_3nonsense.head()

,sequence,switched,first_word,second_word,final_word
0,The baz hit the fuu. The fuu hit the schleep. ...,False,baz,fuu,schleep
1,The fuu hit the schleep. The baz hit the fuu. ...,True,baz,fuu,schleep
2,The baz hit the fuu. The fuu hit the blubb. Th...,False,baz,fuu,blubb
3,The fuu hit the blubb. The baz hit the fuu. Th...,True,baz,fuu,blubb
4,The baz hit the fuu. The fuu hit the bla. The ...,False,baz,fuu,bla


In [26]:
question1 = "\nQuestion: What started the chain?\nAnswer: "
question2 = "\nQuestion: What was second in the chain?\nAnswer: "
question3 = "\nQuestion: What fell into the hole?\nAnswer: "

In [27]:
# run and populate the entire dataframe
answers_n3_q1 = []
answers_n3_q2 = []
answers_n3_q3 = []

for index, row in tqdm(df_toy_problem_3nonsense.iterrows()):
    prompt_q1 = row["sequence"] +  question1
    prompt_q2 = row["sequence"] +  question2
    prompt_q3 = row["sequence"] +  question3
    response_q1 = openai.Completion.create(engine=gpt3_model, prompt=prompt_q1, max_tokens=15, temperature=0)
    response_text_q1 = response_q1["choices"][0]["text"]
    response_q2 = openai.Completion.create(engine=gpt3_model, prompt=prompt_q2, max_tokens=15, temperature=0)
    response_text_q2 = response_q2["choices"][0]["text"]
    response_q3 = openai.Completion.create(engine=gpt3_model, prompt=prompt_q3, max_tokens=15, temperature=0)
    response_text_q3 = response_q3["choices"][0]["text"]
#    print(response_text_q1)
#    print(response_text_q2)
#    print(response_text_q3)
    answers_n3_q1.append(response_text_q1)
    answers_n3_q2.append(response_text_q2)
    answers_n3_q3.append(response_text_q3)

70it [02:08,  1.83s/it]


In [28]:
df_toy_problem_3nonsense["answers_start_chain"] = answers_n3_q1
df_toy_problem_3nonsense["answers_second_chain"] = answers_n3_q2
df_toy_problem_3nonsense["answers_into_hole"] = answers_n3_q3

path = "data/toy_problem_3_results/toy_problem_3nonsense_results_{}.csv".format(gpt3_model)
df_toy_problem_3nonsense.to_csv(path, index=False)
df_toy_problem_3nonsense.head()

,sequence,switched,first_word,second_word,final_word,answers_start_chain,answers_second_chain,answers_into_hole
0,The baz hit the fuu. The fuu hit the schleep. ...,False,baz,fuu,schleep,\n\nThe baz hit the fuu.,\n\nThe hole.,\n\nThe schleep fell in the hole.
1,The fuu hit the schleep. The baz hit the fuu. ...,True,baz,fuu,schleep,\n\nThe baz hit the fuu.,\n\nThe baz hit the fuu.,\n\nThe fuu hit the schleep. The baz hit the
2,The baz hit the fuu. The fuu hit the blubb. Th...,False,baz,fuu,blubb,\n\nThe baz hit the fuu.,\n\nThe blubb.,\n\nThe blubb fell in the hole.
3,The fuu hit the blubb. The baz hit the fuu. Th...,True,baz,fuu,blubb,\n\nThe fuu hit the blubb.,\n\nThe baz hit the fuu.,\n\nThe blubb fell in the hole.
4,The baz hit the fuu. The fuu hit the bla. The ...,False,baz,fuu,bla,\n\nThe baz hit the fuu.,\n\nThe bla fell in the hole.,\n\nThe bla fell in the hole.
